- quickstart: go through installation, imports, etc. 
- understand why to use cuML
- at a high level, what are main features of cuML and why you would use it
- put in links to documentation (cuML + scikitlearn) 
- explain what is happening in each example 

# 🚀 Welcome to the World of cuML!

As part of the NVIDIA RAPIDS suite, cuML is incredibly useful for accelerating the end-to-end machine learning pipeline, from data preprocessing to model training and evaluation, utilizing the parallel processing capabilities of NVIDIA GPUs. It’s like taking a turbocharged sports car for a spin instead of a regular sedan—everything is faster and more exciting!



To sum it up, core strengths of cuML include: 
🎉 matches scikit-learn API (users are already familiar with syntax and functionalities)
🎉 cuML estimators accept flexible types and returns predictable types
🎉 can optimize CPU based equivalents by a factor of 4x to 100x





## 📊 Let's Dive Into Data
cuML works seamlessly with cuDF DataFrames, so let’s create a simple DataFrame.

In [2]:
import cudf
import numpy as np

# Creating a random DataFrame
data = cudf.DataFrame({
    'x1': np.random.rand(1000),
    'x2': np.random.rand(1000),
    'y': np.random.randint(0, 2, size=1000)
})

data.head()

,x1,x2,y
0,0.256883,0.828882,0
1,0.410634,0.166474,1
2,0.841882,0.942280,0
3,0.574056,0.059603,1
4,0.345641,0.234410,0


💡 Challenge: modify the number of rows in the DataFrame and observe how it changes the output!


## 📈 Training a Model
Let’s train a simple Logistic Regression model to understand how cuML speeds up the training process.

In [12]:
from cuml.linear_model import LogisticRegression
from cuml.model_selection import train_test_split

#split the data into training and testing sets
X = data[['x1', 'x2']]
y = data['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8)
model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

💡 Challenge: Play around with different parameters for LogisticRegression. Try changing the solver or adding regularization. What happens?

## 🔍 Evaluating Your Model

### Evaluate the model's performance using accuracy.

In [ ]:

from cuml.metrics.accuracy import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy:.4f}")


## 📊 Hyperparameter Tuning with Grid Search
### We can perform hyperparameter tuning using cuML's GridSearchCV.


In [ ]:

from cuml.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'fit_intercept': [True, False],
    'max_iter': [100, 200]
}

grid_search = GridSearchCV(model, param_grid, scoring='accuracy', cv=3)
grid_search.fit(X_train, y_train)

# Get the best parameters and score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_:.4f}")

## 📊 Cross-Validation

### Perform cross-validation to get a better estimate of the model performance.

In [ ]:
from cuml.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5)
print(f"Cross-Validation Scores: {scores}")
print(f"Mean Cross-Validation Score: {scores.mean():.4f}")

## 🔍 Comparing GPU & CPU Models

How long did fitting the model on the GPU take?

In [9]:
import time 

# Timing the GPU model training
start_time = time.time()
model = LogisticRegression()
model.fit(X_train, y_train)
gpu_time = time.time() - start_time
print(f"GPU Training Time: {gpu_time:.4f} seconds")

GPU Training Time: 0.0118 seconds


Lets do the same for a model fit on the CPU instead: 

In [10]:
from sklearn.linear_model import LogisticRegression as SklearnLogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd

# Create a large random DataFrame using pandas
data_pd = pd.DataFrame({
    'x1': np.random.rand(1000000),
    'x2': np.random.rand(1000000),
    'y': np.random.randint(0, 2, size=1000000)
})

X_pd = data_pd[['x1', 'x2']]
y_pd = data_pd['y']
X_train_pd, X_test_pd, y_train_pd, y_test_pd = train_test_split(X_pd, y_pd, test_size=0.2)


# Timing the CPU model training
start_time = time.time()
cpu_model = SklearnLogisticRegression()
cpu_model.fit(X_train_pd, y_train_pd)

cpu_time = time.time() - start_time
print(f"CPU Training Time: {cpu_time:.4f} seconds")



CPU Training Time: 1.0668 seconds


Although on surface level the code looked almost identical, the model using cuML was almost 100x as fast as the one using just scikitlearn! cuML on a GPU can significantly outperform traditional CPU-based machine learning libraries, especially with large datasets. The time savings become more pronounced as the data size increases, showcasing the advantages of leveraging GPU acceleration for machine learning tasks.

💡 Challenge: if you have a specific dataset or model in mind, you could run the above examples to see the time differences firsthand!


### Saving & Loading Your Model


In [ ]:
!pip install joblib

In [ ]:
# Save the trained model using cuML's joblib.

import joblib

joblib.dump(model, 'logistic_regression_model.pkl')

# Load the model
loaded_model = joblib.load('logistic_regression_model.pkl')

## Additional Examples 


### K-Means Clustering

In [ ]:
# 📊 Using K-Means Clustering
# Let's demonstrate K-Means clustering using cuML.

from cuml.cluster import KMeans
from cupy import cp 

# Generate synthetic data for clustering
X_clustering = cp.random.rand(10000, 2)  # 10,000 samples, 2 features

# Initialize KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(X_clustering)

# Predict cluster labels
labels = kmeans.predict(X_clustering)

# Display the first few labels
print(labels[:10])


### PCA for Dimensionality Reduction 

In [ ]:
# 📊 PCA for Dimensionality Reduction
# Perform PCA on the dataset for dimensionality reduction.

from cuml.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

print(f"PCA Transformed Shape: {X_pca.shape}")


## Advanced Features of cuML

### Multi-GPU Support 

cuML supports multi-GPU setups, allowing you to scale your computations further. This is especially useful for extremely large datasets or complex models that benefit from distributed processing.

### Integration wth Other RAPIDS Libraries 

cuML integrates well with other RAPIDS libraries like cuDF (for data manipulation), cuGraph (for graph analytics), and cuSpatial (for spatial data). This synergy allows you to build comprehensive data science workflows entirely on the GPU.


### Use Cases

- Big Data Analytics: Ideal for applications requiring the processing of large volumes of data, such as financial analysis or real-time analytics.
- Deep Learning Preprocessing: Use cuML for preprocessing steps in machine learning workflows, significantly reducing bottlenecks when training deep learning models.
- Time Series Forecasting: Speed up training on time series models that involve massive datasets.

## Conclusion 

cuML is a game-changer for machine learning practitioners looking to harness the power of GPUs. By providing a familiar interface, significant performance improvements, and seamless integration with existing workflows, cuML enables faster iterations and more efficient model training.

### 🌟 Final Thoughts

As you explore cuML, think about the specific challenges you face in your machine learning projects. How could GPU acceleration improve your workflows? With cuML, you can unlock new levels of performance and efficiency in your data science endeavors!

